#   ETL for prices, sales, quant_fe and ecomm data


In [30]:
import pandas as pd
import numpy as np
import os
import io
import getpass
import psycopg2
from datetime import datetime


In [31]:
# get resource folder path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
resource_dir = os.path.join(parent_dir, 'Resources')


##   STEP 1 - IMPORT AND PREPARE THE DATA


In [68]:
# load csv files that need transposing into pandas dataframes
df_prices_raw = pd.read_csv(os.path.join(resource_dir,'data3','prices.csv'))
# df_quant_fe_raw = pd.read_csv(os.path.join(resource_dir,'data3','quant_fe.csv'))
df_sales_raw = pd.read_csv(os.path.join(resource_dir,'data3','sales.csv'))

# import the e-commerce csv file 
df_ecomm = pd.read_csv(os.path.join(resource_dir,'data7','walmart_com-ecommerce_product_details.csv'))


In [72]:
# check for empty string text ids (keys)
# print((df_sales_raw['id'].values == '').sum())
# print((df_prices_raw['id'].values == '').sum())
# print((df_quant_fe_raw['id'].values == '').sum())


In [70]:
# check that all ids in prices and quant are in sales
print('  length of df_prices_raw: ' + str(len(df_prices_raw['id'])))
print('     df_prices_raw unique: ' + str(df_prices_raw['id'].nunique()))

print('\n   length of df_sales_raw: ' + str(len(df_sales_raw['id'])))
print('      df_sales_raw unique: ' + str(df_sales_raw['id'].nunique()))

# print('\nlength of df_quant_fe_raw: ' + str(len(df_sales_raw['id'])))
# print('   df_quant_fe_raw unique: ' + str(df_sales_raw['id'].nunique()))

df_ids = df_sales_raw.merge(df_prices_raw, how='outer', on='id')
print('\n       sales outer prices: ' + str(df_ids['id'].nunique()))

df_ids = df_sales_raw.merge(df_quant_fe_raw, how='outer', on='id')
print('\n        sales outer quant: ' + str(df_ids['id'].nunique()))


  length of df_prices_raw: 30490
     df_prices_raw unique: 30490

   length of df_sales_raw: 30490
      df_sales_raw unique: 30490

       sales outer prices: 42840

        sales outer quant: 42840


In [35]:
df_sales_raw

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,start,scale
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,3,3,0,1,901,0.773603
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,0,0,0,0,143,0.422049
2,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,0,0,0,2,3,0,1,1105,0.511990
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,4,0,1,3,0,2,6,36,1.778245
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,2,1,0,0,2,1,0,112,1.102354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42835,WI_3_X,X,X,X,WI_3,WI,4038,4198,3317,3211,...,5894,3897,3306,3268,3398,4126,4519,4757,0,538.851470
42836,CA_X,X,X,X,CA,CA,14195,13805,10108,11047,...,24704,17721,16150,15678,16297,17430,23103,24644,0,1970.131511
42837,TX_X,X,X,X,TX,TX,9438,9630,6778,7381,...,16249,12228,11370,10375,9162,12303,13681,14815,0,1349.161939
42838,WI_X,X,X,X,WI,WI,8998,8314,6897,6984,...,18968,12413,11257,11043,11504,12819,14734,14879,0,1440.429603


### Sales and Prices
####     returns:
####         - df_sales_raw (main table with foreign keys, requires transposition)
####         - df_sales_categories (keyed dimension table)
####         - df_sales_departments (keyed dimension table)
####         - df_sales_stores (keyed dimension table)
####         - df_sales_states (keyed dimension table)
####         - df_sales_items (keyed dimension table)
####         - df_prices_raw (main table with foreign keys, requires transposition)


In [36]:
# the sales data has subtotals that should be dropped
df_sales_raw.drop(df_sales_raw[df_sales_raw['item_id'] == 'X'].index, inplace=True)
df_sales_raw.drop(df_sales_raw[df_sales_raw['state_id'] == 'X'].index, inplace=True)
df_sales_raw.drop(df_sales_raw[df_sales_raw['store_id'] == 'X'].index, inplace=True)
df_sales_raw.drop(df_sales_raw[df_sales_raw['cat_id'] == 'X'].index, inplace=True)
df_sales_raw.drop(df_sales_raw[df_sales_raw['dept_id'] == 'X'].index, inplace=True)

df_sales_raw.drop(df_sales_raw[df_sales_raw['id'].astype('str').str[:2] == 'WI'].index, inplace=True)
df_sales_raw.drop(df_sales_raw[df_sales_raw['id'].astype('str').str[:2] == 'TX'].index, inplace=True)
df_sales_raw.drop(df_sales_raw[df_sales_raw['id'].astype('str').str[:2] == 'CA'].index, inplace=True)

df_sales_raw

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,start,scale
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,3,3,0,1,901,0.773603
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,0,0,0,0,143,0.422049
2,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,0,0,0,2,3,0,1,1105,0.511990
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,4,0,1,3,0,2,6,36,1.778245
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,2,1,0,0,2,1,0,112,1.102354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,3,0,1,1,0,0,1,1,2,0.578730
30486,FOODS_3_824_WI_3,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,1,0,1,0,5,0.485005
30487,FOODS_3_825_WI_3,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,1,2,0,1,0,1,0,2,1,0.872033
30488,FOODS_3_826_WI_3,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,4,6,0,1,1,1,0,939,0.806000


In [37]:
# create category data from the prices/sales/quant data
df_sales_id = df_sales_raw.groupby('id').size().reset_index()
df_sales_id = df_sales_id.loc[:, ['id']]
df_sales_id.rename(columns = {'id': 'id_desc'}, inplace=True) 
df_sales_id['id'] = df_sales_id.index

df_sales_categories = df_sales_raw[['cat_id']].dropna(axis=0).groupby('cat_id').size().reset_index()
df_sales_categories = df_sales_categories.loc[:, ['cat_id']]
df_sales_categories.rename(columns = {'cat_id': 'cat_desc'}, inplace=True) 
df_sales_categories['cat_id'] = df_sales_categories.index

df_sales_departments = df_sales_raw[['dept_id']].dropna(axis=0).groupby('dept_id').size().reset_index()
df_sales_departments = df_sales_departments.loc[:, ['dept_id']]
df_sales_departments.rename(columns = {'dept_id': 'dept_desc'}, inplace=True) 
df_sales_departments['dept_id'] = df_sales_departments.index

df_sales_stores = df_sales_raw[['store_id']].dropna(axis=0).groupby('store_id').size().reset_index()
df_sales_stores = df_sales_stores.loc[:, ['store_id']]
df_sales_stores.rename(columns = {'store_id': 'store_desc'}, inplace=True) 
df_sales_stores['store_id'] = df_sales_stores.index 

# df_sales_states = df_sales_raw[['state_id']].dropna(axis=0).groupby('state_id').size().reset_index()
# df_sales_states = df_sales_states.loc[:, ['state_id']]
# df_sales_states.rename(columns = {'state_id': 'state_desc'}, inplace=True) 
# df_sales_states['state_id'] = df_sales_states.index 

# manually create the states dataframe - to align the keys with other tables being used!
states = [[6,'CA'],[48,'TX'],[55,'WI']]
df_sales_states = pd.DataFrame(states, columns = ['state_id', 'state_desc'])

df_sales_items = df_sales_raw[['item_id']].dropna(axis=0).groupby('item_id').size().reset_index()
df_sales_items = df_sales_items.loc[:, ['item_id']]
df_sales_items.rename(columns = {'item_id': 'item_desc'}, inplace=True) 
df_sales_items['item_id'] = df_sales_items.index


In [38]:
df_sales_id

,id_desc,id
0,FOODS_1_001_CA_1,0
1,FOODS_1_001_CA_2,1
2,FOODS_1_001_CA_3,2
3,FOODS_1_001_CA_4,3
4,FOODS_1_001_TX_1,4
...,...,...
30485,HOUSEHOLD_2_516_TX_2,30485
30486,HOUSEHOLD_2_516_TX_3,30486
30487,HOUSEHOLD_2_516_WI_1,30487
30488,HOUSEHOLD_2_516_WI_2,30488


In [39]:
df_sales_categories

,cat_desc,cat_id
0,FOODS,0
1,HOBBIES,1
2,HOUSEHOLD,2


In [40]:
df_sales_departments

,dept_desc,dept_id
0,FOODS_1,0
1,FOODS_2,1
2,FOODS_3,2
3,HOBBIES_1,3
4,HOBBIES_2,4
5,HOUSEHOLD_1,5
6,HOUSEHOLD_2,6


In [41]:
df_sales_stores

,store_desc,store_id
0,CA_1,0
1,CA_2,1
2,CA_3,2
3,CA_4,3
4,TX_1,4
5,TX_2,5
6,TX_3,6
7,WI_1,7
8,WI_2,8
9,WI_3,9


In [42]:
df_sales_states

,state_id,state_desc
0,6,CA
1,48,TX
2,55,WI


In [43]:
# add the categories to prices 
# since we're removing the daily sales data with 0 sales, we need to have all the id for the prices table
# so the table can be further summarized
df_prices_raw = df_prices_raw.merge(df_sales_raw[['id','dept_id','cat_id','item_id','store_id','state_id']], how='left', left_on='id', right_on='id')
df_prices_raw


,id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1965,d_1966,d_1967,d_1968,d_1969,dept_id,cat_id,item_id,store_id,state_id
0,FOODS_1_001_CA_1,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,FOODS_1,FOODS,FOODS_1_001,CA_1,CA
1,FOODS_1_001_CA_2,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,FOODS_1,FOODS,FOODS_1_001,CA_2,CA
2,FOODS_1_001_CA_3,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,FOODS_1,FOODS,FOODS_1_001,CA_3,CA
3,FOODS_1_001_CA_4,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,FOODS_1,FOODS,FOODS_1_001,CA_4,CA
4,FOODS_1_001_TX_1,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,FOODS_1,FOODS,FOODS_1_001,TX_1,TX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,5.94,5.94,5.94,5.94,5.94,HOUSEHOLD_2,HOUSEHOLD,HOUSEHOLD_2_516,TX_2,TX
30486,HOUSEHOLD_2_516_TX_3,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,5.94,5.94,5.94,5.94,5.94,HOUSEHOLD_2,HOUSEHOLD,HOUSEHOLD_2_516,TX_3,TX
30487,HOUSEHOLD_2_516_WI_1,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,5.94,5.94,5.94,5.94,5.94,HOUSEHOLD_2,HOUSEHOLD,HOUSEHOLD_2_516,WI_1,WI
30488,HOUSEHOLD_2_516_WI_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.94,5.94,5.94,5.94,5.94,HOUSEHOLD_2,HOUSEHOLD,HOUSEHOLD_2_516,WI_2,WI


In [44]:
# confirm no nulls in new dataframes
print('nulls in df_sales_id\n' + str(len(df_sales_id) - df_sales_id.count()))
print('\nnulls in df_sales_categories\n' + str(len(df_sales_categories) - df_sales_categories.count()))
print('\nnulls in df_sales_departments\n' + str(len(df_sales_departments) - df_sales_departments.count()))
print('\nnulls in df_sales_stores\n' + str(len(df_sales_stores) - df_sales_stores.count()))
print('\nnulls in df_sales_states\n' + str(len(df_sales_states) - df_sales_states.count()))
print('\nnulls in df_sales_items\n' + str(len(df_sales_items) - df_sales_items.count()))

nulls in df_sales_id
id_desc    0
id         0
dtype: int64

nulls in df_sales_categories
cat_desc    0
cat_id      0
dtype: int64

nulls in df_sales_departments
dept_desc    0
dept_id      0
dtype: int64

nulls in df_sales_stores
store_desc    0
store_id      0
dtype: int64

nulls in df_sales_states
state_id      0
state_desc    0
dtype: int64

nulls in df_sales_items
item_desc    0
item_id      0
dtype: int64


In [45]:
# add the foreign keys for categories, departments, items, states and stores to the ***sales*** table
# rename the previous non-numeric ids (delete them after)
df_sales_raw.rename(columns = {'id': 'id_old','cat_id': 'cat_id_old','dept_id': 'dept_id_old','item_id': 'item_id_old', 'state_id': 'state_id_old','store_id': 'store_id_old'}, inplace=True)

# merge df_sales (left) with df_sales_id on id_old = id_desc to get the new numeric id foreign key
df_sales_raw = df_sales_raw.merge(df_sales_id, how='left', left_on='id_old', right_on='id_desc')
df_sales_raw['id'] = df_sales_raw['id'].astype('Int64')

# merge df_sales (left) with df_sales_categories on cat_id_old = cat_id_desc to get the new numeric cat_id foreign key
df_sales_raw = df_sales_raw.merge(df_sales_categories, how='left', left_on='cat_id_old', right_on='cat_desc')
df_sales_raw['cat_id'] = df_sales_raw['cat_id'].astype('Int64')

# merge df_sales (left) with df_sales_departments on dept_id_old = dept_id_desc to get the new numeric dept_id foreign key
df_sales_raw = df_sales_raw.merge(df_sales_departments, how='left', left_on='dept_id_old', right_on='dept_desc')
df_sales_raw['dept_id'] = df_sales_raw['dept_id'].astype('Int64')

# merge df_sales (left) with df_sales_items on item_id_old = item_id_desc to get the new numeric item_id foreign key
df_sales_raw = df_sales_raw.merge(df_sales_items, how='left', left_on='item_id_old', right_on='item_desc')
df_sales_raw['item_id'] = df_sales_raw['item_id'].astype('Int64')

# merge df_sales (left) with df_sales_states on state_id_old = state_id_desc to get the new numeric state_id foreign key
df_sales_raw = df_sales_raw.merge(df_sales_states, how='left', left_on='state_id_old', right_on='state_desc')
df_sales_raw['state_id'] = df_sales_raw['state_id'].astype('Int64')

# merge df_sales (left) with df_sales_stores on store_id_old = store_id_desc to get the new numeric store_id foreign key
df_sales_raw = df_sales_raw.merge(df_sales_stores, how='left', left_on='store_id_old', right_on='store_desc')
df_sales_raw['store_id'] = df_sales_raw['store_id'].astype('Int64')

df_sales_raw

,id_old,item_id_old,dept_id_old,cat_id_old,store_id_old,state_id_old,d_1,d_2,d_3,d_4,...,cat_desc,cat_id,dept_desc,dept_id,item_desc,item_id,state_id,state_desc,store_desc,store_id
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,HOBBIES,1,HOBBIES_1,3,HOBBIES_1_001,1437,6,CA,CA_1,0
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,HOBBIES,1,HOBBIES_1,3,HOBBIES_1_002,1438,6,CA,CA_1,0
2,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,HOBBIES,1,HOBBIES_1,3,HOBBIES_1_003,1439,6,CA,CA_1,0
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,HOBBIES,1,HOBBIES_1,3,HOBBIES_1_004,1440,6,CA,CA_1,0
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,HOBBIES,1,HOBBIES_1,3,HOBBIES_1_005,1441,6,CA,CA_1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,FOODS,0,FOODS_3,2,FOODS_3_823,1432,55,WI,WI_3,9
30486,FOODS_3_824_WI_3,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,FOODS,0,FOODS_3,2,FOODS_3_824,1433,55,WI,WI_3,9
30487,FOODS_3_825_WI_3,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,FOODS,0,FOODS_3,2,FOODS_3_825,1434,55,WI,WI_3,9
30488,FOODS_3_826_WI_3,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,FOODS,0,FOODS_3,2,FOODS_3_826,1435,55,WI,WI_3,9


In [46]:
# add the foreign keys for categories, departments, items, states and stores to the ***prices*** table
# rename the previous non-numeric ids (delete them after)
df_prices_raw.rename(columns = {'id': 'id_old','cat_id': 'cat_id_old','dept_id': 'dept_id_old','item_id': 'item_id_old','state_id': 'state_id_old','store_id': 'store_id_old'}, inplace=True)

# merge df_prices (left) with df_sales_id on id_old = id_desc to get the new numeric id foreign key
df_prices_raw = df_prices_raw.merge(df_sales_id, how='left', left_on='id_old', right_on='id_desc')
df_prices_raw['id'] = df_prices_raw['id'].astype('Int64')

# merge df_prices (left) with df_sales_categories on cat_id_old = cat_id_desc to get the new numeric cat_id foreign key
df_prices_raw = df_prices_raw.merge(df_sales_categories, how='left', left_on='cat_id_old', right_on='cat_desc')
df_prices_raw['cat_id'] = df_prices_raw['cat_id'].astype('Int64')

# merge df_prices (left) with df_sales_departments on dept_id_old = dept_id_desc to get the new numeric dept_id foreign key
df_prices_raw = df_prices_raw.merge(df_sales_departments, how='left', left_on='dept_id_old', right_on='dept_desc')
df_prices_raw['dept_id'] = df_prices_raw['dept_id'].astype('Int64')

# merge df_prices (left) with df_sales_items on item_id_old = item_id_desc to get the new numeric item_id foreign key
df_prices_raw = df_prices_raw.merge(df_sales_items, how='left', left_on='item_id_old', right_on='item_desc')
df_prices_raw['item_id'] = df_prices_raw['item_id'].astype('Int64')

# merge df_prices (left) with df_sales_states on state_id_old = state_id_desc to get the new numeric state_id foreign key
df_prices_raw = df_prices_raw.merge(df_sales_states, how='left', left_on='state_id_old', right_on='state_desc')
df_prices_raw['state_id'] = df_prices_raw['state_id'].astype('Int64')

# merge df_prices (left) with df_sales_stores on store_id_old = store_id_desc to get the new numeric store_id foreign key
df_prices_raw = df_prices_raw.merge(df_sales_stores, how='left', left_on='store_id_old', right_on='store_desc')
df_prices_raw['store_id'] = df_prices_raw['store_id'].astype('Int64')

df_prices_raw

,id_old,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,cat_desc,cat_id,dept_desc,dept_id,item_desc,item_id,state_id,state_desc,store_desc,store_id
0,FOODS_1_001_CA_1,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,FOODS,0,FOODS_1,0,FOODS_1_001,0,6,CA,CA_1,0
1,FOODS_1_001_CA_2,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,FOODS,0,FOODS_1,0,FOODS_1_001,0,6,CA,CA_2,1
2,FOODS_1_001_CA_3,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,FOODS,0,FOODS_1,0,FOODS_1_001,0,6,CA,CA_3,2
3,FOODS_1_001_CA_4,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,FOODS,0,FOODS_1,0,FOODS_1_001,0,6,CA,CA_4,3
4,FOODS_1_001_TX_1,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,FOODS,0,FOODS_1,0,FOODS_1_001,0,48,TX,TX_1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,HOUSEHOLD,2,HOUSEHOLD_2,6,HOUSEHOLD_2_516,3048,48,TX,TX_2,5
30486,HOUSEHOLD_2_516_TX_3,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,HOUSEHOLD,2,HOUSEHOLD_2,6,HOUSEHOLD_2_516,3048,48,TX,TX_3,6
30487,HOUSEHOLD_2_516_WI_1,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,HOUSEHOLD,2,HOUSEHOLD_2,6,HOUSEHOLD_2_516,3048,55,WI,WI_1,7
30488,HOUSEHOLD_2_516_WI_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,HOUSEHOLD,2,HOUSEHOLD_2,6,HOUSEHOLD_2_516,3048,55,WI,WI_2,8


In [47]:
# drop old non-numeric ids and redundant merged descriptions
df_sales_raw.drop(['id_old','id_desc','cat_id_old','cat_desc','dept_id_old','dept_desc','item_id_old','item_desc','state_id_old','state_desc','store_id_old','store_desc'],axis=1,inplace=True)
df_sales_raw


,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1940,d_1941,start,scale,id,cat_id,dept_id,item_id,state_id,store_id
0,0,0,0,0,0,0,0,0,0,0,...,0,1,901,0.773603,14370,1,3,1437,6,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,143,0.422049,14380,1,3,1438,6,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1105,0.511990,14390,1,3,1439,6,0
3,0,0,0,0,0,0,0,0,0,0,...,2,6,36,1.778245,14400,1,3,1440,6,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,112,1.102354,14410,1,3,1441,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,1,1,2,0.578730,14329,0,2,1432,55,9
30486,0,0,0,0,0,5,0,1,1,3,...,1,0,5,0.485005,14339,0,2,1433,55,9
30487,0,6,0,2,2,4,1,8,5,2,...,0,2,1,0.872033,14349,0,2,1434,55,9
30488,0,0,0,0,0,0,0,0,0,0,...,1,0,939,0.806000,14359,0,2,1435,55,9


In [48]:
# drop old non-numeric ids and redundant merged descriptions

df_prices_raw.drop('id_old',axis=1,inplace=True)
df_prices_raw.drop('id_desc',axis=1,inplace=True)

df_prices_raw.drop('cat_id_old',axis=1,inplace=True)
df_prices_raw.drop('cat_desc',axis=1,inplace=True)

df_prices_raw.drop('dept_id_old',axis=1,inplace=True)
df_prices_raw.drop('dept_desc',axis=1,inplace=True)

df_prices_raw.drop('item_id_old',axis=1,inplace=True)
df_prices_raw.drop('item_desc',axis=1,inplace=True)

df_prices_raw.drop('state_id_old',axis=1,inplace=True)
df_prices_raw.drop('state_desc',axis=1,inplace=True)

df_prices_raw.drop('store_id_old',axis=1,inplace=True)
df_prices_raw.drop('store_desc',axis=1,inplace=True)

df_prices_raw


,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1966,d_1967,d_1968,d_1969,id,cat_id,dept_id,item_id,state_id,store_id
0,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,0,0,0,0,6,0
1,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,1,0,0,0,6,1
2,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2,0,0,0,6,2
3,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,3,0,0,0,6,3
4,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,4,0,0,0,48,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,5.94,5.94,5.94,5.94,30485,2,6,3048,48,5
30486,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,5.94,5.94,5.94,5.94,30486,2,6,3048,48,6
30487,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,5.94,5.94,5.94,5.94,30487,2,6,3048,55,7
30488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.94,5.94,5.94,5.94,30488,2,6,3048,55,8


In [49]:
# the start and scale columns shouldn't be transposed in this table because they're item and store level
# start is not needed because the 0 sales roles will be deleted after being transposed
# the scale should be recalculated if needed on the grouped amounts

df_sales_raw.drop(['start','scale'],axis=1,inplace=True)
df_sales_raw


,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1938,d_1939,d_1940,d_1941,id,cat_id,dept_id,item_id,state_id,store_id
0,0,0,0,0,0,0,0,0,0,0,...,3,3,0,1,14370,1,3,1437,6,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,14380,1,3,1438,6,0
2,0,0,0,0,0,0,0,0,0,0,...,2,3,0,1,14390,1,3,1439,6,0
3,0,0,0,0,0,0,0,0,0,0,...,3,0,2,6,14400,1,3,1440,6,0
4,0,0,0,0,0,0,0,0,0,0,...,0,2,1,0,14410,1,3,1441,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,0,0,1,1,14329,0,2,1432,55,9
30486,0,0,0,0,0,5,0,1,1,3,...,1,0,1,0,14339,0,2,1433,55,9
30487,0,6,0,2,2,4,1,8,5,2,...,0,1,0,2,14349,0,2,1434,55,9
30488,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,14359,0,2,1435,55,9


### quant_fe - DECIDED NOT TO USE
####     returns:
####         - df_quant_fe (main table with id that matches sales and prices)

In [50]:
# # add the foreign key for id to the ***quant_fe*** table
# # delete the previous non-numeric ids
# df_quant_fe_raw.rename(columns = {'id': 'id_old'}, inplace=True)

# # merge df_sales (left) with df_sales_id on id_old = id_desc to get the new numeric id foreign key
# df_quant_fe_raw = df_quant_fe_raw.merge(df_sales_id, how='left', left_on='id_old', right_on='id_desc')
# df_quant_fe_raw['id'] = df_quant_fe_raw['id'].astype('Int64')
# df_quant_fe_raw

In [51]:
# df_quant_fe_raw.drop('id_old',axis=1,inplace=True)
# df_quant_fe_raw.drop('id_desc',axis=1,inplace=True)
# df_quant_fe_raw

### e-commerce
####     returns:
####         - df_ecomm_prod_cat (main table with foreign keys to the 6 category levels)
####         - df_ecomm_category_level1
####         - df_ecomm_category_level2 - DECIDED NOT TO USE
####         - df_ecomm_category_level3 - DECIDED NOT TO USE
####         - df_ecomm_category_level4 - DECIDED NOT TO USE
####         - df_ecomm_category_level5 - DECIDED NOT TO USE
####         - df_ecomm_category_level6 - DECIDED NOT TO USE

In [52]:
# # create category data from the ecomm data
# df_ecomm_prod_cat = df_ecomm[['Category']].dropna(axis=0).groupby('Category').size().reset_index()
# df_ecomm_prod_cat = df_ecomm_prod_cat.loc[:, ['Category']]
# df_ecomm_prod_cat.rename(columns = {'Category': 'category_combined_desc'}, inplace=True) 
# df_ecomm_prod_cat.index.name = 'category_id'

# # split the combined category column into its 6 levels on the '_' in the combined name
# df_ecomm_prod_cat[['category_level1','category_level2','category_level3','category_level4','category_level5','category_level6']] = df_ecomm_prod_cat['category_combined_desc'].str.split('|',5,expand=True)

# category_level1
# df_ecomm_category_level1 = df_ecomm_prod_cat[['category_level1']].dropna(axis=0).groupby('category_level1').size().reset_index()
# df_ecomm_category_level1 = df_ecomm_category_level1.loc[:, ['category_level1']]
# df_ecomm_category_level1.rename(columns = {'category_level1': 'category_level1_desc'}, inplace=True) 
# df_ecomm_category_level1['category_level1_id'] = df_ecomm_category_level1.index

# # category_level2
# df_ecomm_category_level2 = df_ecomm_prod_cat[['category_level2']].dropna(axis=0).groupby('category_level2').size().reset_index()
# df_ecomm_category_level2 = df_ecomm_category_level2.loc[:, ['category_level2']]
# df_ecomm_category_level2.rename(columns = {'category_level2': 'category_level2_desc'}, inplace=True) 
# df_ecomm_category_level2['category_level2_id'] = df_ecomm_category_level2.index

# # category_level3
# df_ecomm_category_level3 = df_ecomm_prod_cat[['category_level3']].dropna(axis=0).groupby('category_level3').size().reset_index()
# df_ecomm_category_level3 = df_ecomm_category_level3.loc[:, ['category_level3']]
# df_ecomm_category_level3.rename(columns = {'category_level3': 'category_level3_desc'}, inplace=True) 
# df_ecomm_category_level3['category_level3_id'] = df_ecomm_category_level3.index

# # category_level4
# df_ecomm_category_level4 = df_ecomm_prod_cat[['category_level4']].dropna(axis=0).groupby('category_level4').size().reset_index()
# df_ecomm_category_level4 = df_ecomm_category_level4.loc[:, ['category_level4']]
# df_ecomm_category_level4.rename(columns = {'category_level4': 'category_level4_desc'}, inplace=True) 
# df_ecomm_category_level4['category_level4_id'] = df_ecomm_category_level4.index

# # category_level5
# df_ecomm_category_level5 = df_ecomm_prod_cat[['category_level5']].dropna(axis=0).groupby('category_level5').size().reset_index()
# df_ecomm_category_level5 = df_ecomm_category_level5.loc[:, ['category_level5']]
# df_ecomm_category_level5.rename(columns = {'category_level5': 'category_level5_desc'}, inplace=True) 
# df_ecomm_category_level5['category_level5_id'] = df_ecomm_category_level5.index

# # category_level6
# df_ecomm_category_level6 = df_ecomm_prod_cat[['category_level6']].dropna(axis=0).groupby('category_level6').size().reset_index()
# df_ecomm_category_level6 = df_ecomm_category_level6.loc[:, ['category_level6']]
# df_ecomm_category_level6.rename(columns = {'category_level6': 'category_level6_desc'}, inplace=True) 
# df_ecomm_category_level6['category_level6_id'] = df_ecomm_category_level6.index


In [53]:
# # rename the previous non-numeric ids (delete them after)
# df_ecomm_prod_cat = df_ecomm_prod_cat.merge(df_ecomm_category_level1, how='left', left_on='category_level1', right_on='category_level1_desc')
# df_ecomm_prod_cat['category_level1_id'] = df_ecomm_prod_cat['category_level1_id'].astype('Int64')

# df_ecomm_prod_cat = df_ecomm_prod_cat.merge(df_ecomm_category_level2, how='left', left_on='category_level2', right_on='category_level2_desc')
# df_ecomm_prod_cat['category_level2_id'] = df_ecomm_prod_cat['category_level2_id'].astype('Int64')

# df_ecomm_prod_cat = df_ecomm_prod_cat.merge(df_ecomm_category_level3, how='left', left_on='category_level3', right_on='category_level3_desc')
# df_ecomm_prod_cat['category_level3_id'] = df_ecomm_prod_cat['category_level3_id'].astype('Int64')

# df_ecomm_prod_cat = df_ecomm_prod_cat.merge(df_ecomm_category_level4, how='left', left_on='category_level4', right_on='category_level4_desc')
# df_ecomm_prod_cat['category_level4_id'] = df_ecomm_prod_cat['category_level4_id'].astype('Int64')

# df_ecomm_prod_cat = df_ecomm_prod_cat.merge(df_ecomm_category_level5, how='left', left_on='category_level5', right_on='category_level5_desc')
# df_ecomm_prod_cat['category_level5_id'] = df_ecomm_prod_cat['category_level5_id'].astype('Int64')

# df_ecomm_prod_cat = df_ecomm_prod_cat.merge(df_ecomm_category_level6, how='left', left_on='category_level6', right_on='category_level6_desc')
# df_ecomm_prod_cat['category_level6_id'] = df_ecomm_prod_cat['category_level6_id'].astype('Int64')

# df_ecomm_prod_cat

In [54]:
# delete the merged descriptions and original descriptions, leaving the ids
# df_ecomm_prod_cat.drop('category_combined_desc',axis=1,inplace=True)

# df_ecomm_prod_cat.drop('category_level1',axis=1,inplace=True)
# df_ecomm_prod_cat.drop('category_level1_desc',axis=1,inplace=True)

# df_ecomm_prod_cat.drop('category_level2',axis=1,inplace=True)
# df_ecomm_prod_cat.drop('category_level2_desc',axis=1,inplace=True)

# df_ecomm_prod_cat.drop('category_level3',axis=1,inplace=True)
# df_ecomm_prod_cat.drop('category_level3_desc',axis=1,inplace=True)

# df_ecomm_prod_cat.drop('category_level4',axis=1,inplace=True)
# df_ecomm_prod_cat.drop('category_level4_desc',axis=1,inplace=True)

# df_ecomm_prod_cat.drop('category_level5',axis=1,inplace=True)
# df_ecomm_prod_cat.drop('category_level5_desc',axis=1,inplace=True)

# df_ecomm_prod_cat.drop('category_level6',axis=1,inplace=True)
# df_ecomm_prod_cat.drop('category_level6_desc',axis=1,inplace=True)

# df_ecomm_prod_cat['category_comb_id'] = df_ecomm_prod_cat.index 

# df_ecomm_prod_cat

In [55]:
# confirm no nulls in new dataframes
# print('nulls in df_ecomm_category_level1\n' + str(len(df_ecomm_category_level1) - df_ecomm_category_level1.count()))
# print('\nnulls in df_ecomm_category_level2\n' + str(len(df_ecomm_category_level2) - df_ecomm_category_level2.count()))
# print('\nnulls in df_ecomm_category_level3\n' + str(len(df_ecomm_category_level3) - df_ecomm_category_level3.count()))
# print('\nnulls in df_ecomm_category_level4\n' + str(len(df_ecomm_category_level4) - df_ecomm_category_level4.count()))
# print('\nnulls in df_ecomm_category_level5\n' + str(len(df_ecomm_category_level5) - df_ecomm_category_level5.count()))
# print('\nnulls in df_ecomm_category_level6\n' + str(len(df_ecomm_category_level6) - df_ecomm_category_level6.count()))

In [74]:
# Split the Categories
df_ecomm[['category_level1','category_level2','category_level3','category_level4','category_level5','category_level6']] = df_ecomm['Category'].str.split('|',5,expand=True)

# filter out everything that isn't available
df_ecomm.drop(df_ecomm[df_ecomm['Available'] == False].index, inplace=True)

# drop unneeded fields: URL, Name, Description, Brand, Package, Size, Postal Code, GTIN, Crawl Time
df_ecomm.drop(['Uniq Id','Crawl Timestamp','Product Url','Product Name','Description','Brand','Item Number','Gtin','Package Size','Postal Code','Category','category_level2','category_level3','category_level4','category_level5','category_level6','Available'],axis=1, inplace=True)

df_ecomm['ecomm_id'] = df_ecomm.index 

df_ecomm

,List Price,Sale Price,category_level1,ecomm_id
0,19.99,19.99,Health,0
1,19.99,19.99,Health,1
2,19.99,19.99,Health,2
3,19.99,19.99,Health,3
7,32.51,32.51,Household Essentials,7
...,...,...,...,...
29995,19.99,19.99,Electronics,29995
29996,19.99,19.99,Electronics,29996
29997,19.99,19.99,Electronics,29997
29998,19.99,19.99,Electronics,29998


# STEP 2 - TRANSFORM THE DATA

In [57]:
# transform ***sales*** dataframe (column sales data into rows and add a 'days' field)
for i in range(1, 1942):
    col_name = 'd_' + str(i)
    df_sales_slice = df_sales_raw.loc[:, ['id','item_id','dept_id','cat_id','store_id','state_id',col_name]]
    
    # drop the rows with 0 sales from the slice
    df_sales_slice.drop(df_sales_slice[df_sales_slice[col_name] == 0].index, inplace=True)
    
    # rename the 'd_xxxx' column to sales
    df_sales_slice.rename(columns = {col_name: 'sales'}, inplace = True)
    df_sales_slice['day'] = i
    
    # if it is not the first column, append (i.e. stack) then data; 
    # if it is the first column, create the dataframe  
    if i != 1:
        df_sales = df_sales.append(df_sales_slice, ignore_index=True)
    else:
        df_sales = df_sales_slice
        
    # give a status update every 100 columns processed
    if i % 100 == 0:
        print(str(i) + ' columns processed...')

df_sales


100 columns processed...
200 columns processed...
300 columns processed...
400 columns processed...
500 columns processed...
600 columns processed...
700 columns processed...
800 columns processed...
900 columns processed...
1000 columns processed...
1100 columns processed...
1200 columns processed...
1300 columns processed...
1400 columns processed...
1500 columns processed...
1600 columns processed...
1700 columns processed...
1800 columns processed...
1900 columns processed...


,id,item_id,dept_id,cat_id,store_id,state_id,sales,day
0,14440,1444,3,1,0,6,12,1
1,14450,1445,3,1,0,6,2,1
2,14510,1451,3,1,0,6,4,1
3,14520,1452,3,1,0,6,5,1
4,14580,1458,3,1,0,6,2,1
...,...,...,...,...,...,...,...,...
18939266,14309,1430,2,0,9,55,4,1941
18939267,14319,1431,2,0,9,55,4,1941
18939268,14329,1432,2,0,9,55,1,1941
18939269,14349,1434,2,0,9,55,2,1941


In [58]:
# transform ***price*** dataframe (column price data into rows and add a 'days' field) 
for i in range(1, 1970):
    col_name = 'd_' + str(i)
    df_prices_slice = df_prices_raw.loc[:, ['id','item_id','dept_id','cat_id','store_id','state_id',col_name]]
    
    # drop the rows with 0 or null prices from the slice
    df_prices_slice.drop(df_prices_slice[df_prices_slice[col_name] == 0].index, inplace=True)
    df_prices_slice.dropna(inplace=True)
    
    
    df_prices_slice.rename(columns = {col_name: 'price'}, inplace = True) 
    df_prices_slice['day'] = i
    if i != 1:
        df_prices = df_prices.append(df_prices_slice, ignore_index=True)
    else:
        df_prices = df_prices_slice
    
    # give a status update every 100 columns processed
    if i % 100 == 0:
        print(str(i) + ' columns processed...')

df_prices


100 columns processed...
200 columns processed...
300 columns processed...
400 columns processed...
500 columns processed...
600 columns processed...
700 columns processed...
800 columns processed...
900 columns processed...
1000 columns processed...
1100 columns processed...
1200 columns processed...
1300 columns processed...
1400 columns processed...
1500 columns processed...
1600 columns processed...
1700 columns processed...
1800 columns processed...
1900 columns processed...


,id,item_id,dept_id,cat_id,store_id,state_id,price,day
0,0,0,0,0,0,6,2.00,1
1,1,0,0,0,1,6,2.00,1
2,2,0,0,0,2,6,2.00,1
3,3,0,0,0,3,6,2.00,1
4,4,0,0,0,4,48,2.00,1
...,...,...,...,...,...,...,...,...
47735392,30485,3048,6,2,5,48,5.94,1969
47735393,30486,3048,6,2,6,48,5.94,1969
47735394,30487,3048,6,2,7,55,5.94,1969
47735395,30488,3048,6,2,8,55,5.94,1969


In [59]:
# transform the prices into effective day ranges for any given price by state/store/cat/dept/item
# join the same table to itself on the end date of the range to get the previous price
# create a price increase indicator (0 for down, 1 for up)

# group by on category ids and price to get the min (start) and max (end) date for every price change
df_price_changes_raw = df_prices.groupby(['state_id','store_id','cat_id','dept_id','item_id','price'], level=None).agg({'day':['min','max']}).reset_index()

# flatten the MultiIndex column name
df_price_changes_raw.columns = ['%s%s' % (a, '%s' % b if b is not None else '') for a, b in df_price_changes_raw.columns]

# rename the columns
df_price_changes_raw.rename(columns = {'daymin':'start_day', 'daymax':'end_day'}, inplace=True)

# calculate the end of the previous period (to use to merge the current price/date range to the previous price)
df_price_changes_raw['_prev_end_day'] = df_price_changes_raw['start_day'] - 1

# merge the current price range data to the previous price
df_price_changes = pd.merge(df_price_changes_raw, df_price_changes_raw, how='left', left_on=['state_id','store_id','cat_id','dept_id','item_id','_prev_end_day'], right_on=['state_id','store_id','cat_id','dept_id','item_id','end_day'], suffixes=('','_prev'))

# drop the extra previous price range data that was merged
df_price_changes.drop(['_prev_end_day','start_day_prev','end_day_prev','_prev_end_day_prev'],axis=1,inplace=True)

# sort the data by the category ids and start day
df_price_changes.sort_values(['state_id','store_id','cat_id','dept_id','item_id','start_day'], inplace=True)

# create an indicator for whether the price change was an increase (1) or not (0)
df_price_changes['price_increase_ind'] = np.where(df_price_changes['price']>df_price_changes['price_prev'], 1, 0)

df_price_changes

,state_id,store_id,cat_id,dept_id,item_id,price,start_day,end_day,price_prev,price_increase_ind
0,6,0,0,0,0,2.00,1,581,NaN,0
1,6,0,0,0,0,2.24,582,1969,2.00,1
2,6,0,0,0,1,7.88,1,252,NaN,0
3,6,0,0,0,1,8.88,253,1365,7.88,1
4,6,0,0,0,1,9.48,1366,1969,8.88,1
...,...,...,...,...,...,...,...,...,...,...
84434,55,9,2,6,3046,18.97,148,980,18.47,1
84432,55,9,2,6,3046,17.94,981,1491,18.97,0
84435,55,9,2,6,3046,19.54,1492,1969,17.94,1
84436,55,9,2,6,3047,1.97,1086,1969,NaN,0


In [63]:
# add price to the sales data
# df_sales = pd.merge(df_sales, df_prices, how='inner', left_on=['item_id','dept_id','cat_id','store_id','state_id','day'], right_on=['item_id','dept_id','cat_id','store_id','state_id','day'], suffixes=('','_price'))

# drop the extra price data
df_sales.drop(['id_price'], axis=1, inplace=True)

df_sales


,id,item_id,dept_id,cat_id,store_id,state_id,sales,day,price
0,14440,1444,3,1,0,6,12,1,0.46
1,14450,1445,3,1,0,6,2,1,1.56
2,14510,1451,3,1,0,6,4,1,0.70
3,14520,1452,3,1,0,6,5,1,0.70
4,14580,1458,3,1,0,6,2,1,6.86
...,...,...,...,...,...,...,...,...,...
18939266,14309,1430,2,0,9,55,4,1941,4.98
18939267,14319,1431,2,0,9,55,4,1941,3.98
18939268,14329,1432,2,0,9,55,1,1941,2.98
18939269,14349,1434,2,0,9,55,2,1941,3.98


In [64]:
# # transform ***quantile*** dataframe (column data into rows and add a 'quantile' field) 
# cols = ['Q5','Q25','Q165','Q250','Q500','Q750','Q835','Q975','Q995']
# i = 1
# for col_name in cols:
#     df_quant_fe_slice = df_quant_fe_raw.loc[:, ['id',col_name]]
#     df_quant_fe_slice.rename(columns = {col_name: 'value'}, inplace = True) 
#     df_quant_fe_slice['quantile'] = col_name
#     if i != 1:
#         df_quant_fe = df_quant_fe.append(df_quant_fe_slice, ignore_index=True)
#     else:
#         df_quant_fe = df_quant_fe_slice
#     i = i + 1
    
# df_quant_fe


In [65]:
df_prices.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47735397 entries, 0 to 47735396
Data columns (total 8 columns):
 #   Column    Dtype  
---  ------    -----  
 0   id        Int64  
 1   item_id   Int64  
 2   dept_id   Int64  
 3   cat_id    Int64  
 4   store_id  Int64  
 5   state_id  Int64  
 6   price     float64
 7   day       int64  
dtypes: Int64(6), float64(1), int64(1)
memory usage: 3.1 GB


In [66]:
# df_quant_fe.info(memory_usage='deep')

In [67]:
df_sales.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18939271 entries, 0 to 18939270
Data columns (total 9 columns):
 #   Column    Dtype  
---  ------    -----  
 0   id        Int64  
 1   item_id   Int64  
 2   dept_id   Int64  
 3   cat_id    Int64  
 4   store_id  Int64  
 5   state_id  Int64  
 6   sales     int64  
 7   day       int64  
 8   price     float64
dtypes: Int64(6), float64(1), int64(2)
memory usage: 1.5 GB


## Step 3 - Save/Load dataframes to .csv (to avoid re-running the ETL)

In [75]:
# export the dataframes to csv files so that the timely transformations don't need to be redone
# df_prices.to_csv(os.path.join(resource_dir, 'clean', 'd3_prices.csv'),index=False)
# df_quant_fe.to_csv(os.path.join(resource_dir, 'clean', 'd3_quant_fe.csv'),index=False)
df_sales.to_csv(os.path.join(resource_dir, 'clean', 'd3_sales.csv'),index=False)
df_price_changes.to_csv(os.path.join(resource_dir, 'clean', 'd3_price_changes.csv'),index=False)

df_sales_categories.to_csv(os.path.join(resource_dir, 'clean', 'd3_categories.csv'),index=False)
df_sales_departments.to_csv(os.path.join(resource_dir, 'clean', 'd3_departments.csv'),index=False)
df_sales_stores.to_csv(os.path.join(resource_dir, 'clean', 'd3_stores.csv'),index=False)
df_sales_states.to_csv(os.path.join(resource_dir, 'clean', 'd3_states.csv'),index=False)
df_sales_items.to_csv(os.path.join(resource_dir, 'clean', 'd3_items.csv'),index=False)

df_ecomm.to_csv(os.path.join(resource_dir, 'clean', 'd7_ecomm.csv'),index=False)
# df_ecomm_prod_cat.to_csv(os.path.join(resource_dir, 'clean', 'd7_prod_cat.csv'),index=False)

# df_ecomm_category_level1.to_csv(os.path.join(resource_dir, 'clean', 'd7_cat_lvl1.csv'),index=False)
# df_ecomm_category_level2.to_csv(os.path.join(resource_dir, 'clean', 'd7_cat_lVl2.csv'),index=False)
# df_ecomm_category_level3.to_csv(os.path.join(resource_dir, 'clean', 'd7_cat_lvl3.csv'),index=False)
# df_ecomm_category_level4.to_csv(os.path.join(resource_dir, 'clean', 'd7_cat_lvl4.csv'),index=False)
# df_ecomm_category_level5.to_csv(os.path.join(resource_dir, 'clean', 'd7_cat_lvl5.csv'),index=False)
# df_ecomm_category_level6.to_csv(os.path.join(resource_dir, 'clean', 'd7_cat_lvl6.csv'),index=False)


In [28]:
# To reload the dataframes from here, run this:
# # df_prices = pd.read_csv(os.path.join(resource_dir, 'clean', 'd3_prices.csv'))
# # df_quant_fe = pd.read_csv(os.path.join(resource_dir, 'clean', 'd3_quant_fe.csv'))
# df_sales = pd.read_csv(os.path.join(resource_dir, 'clean', 'd3_sales.csv'))
# df_price_changes = pd.read_csv(os.path.join(resource_dir, 'clean', 'd3_price_changes.csv'))

# df_sales_categories = pd.read_csv(os.path.join(resource_dir, 'clean', 'd3_categories.csv'))
# df_sales_departments = pd.read_csv(os.path.join(resource_dir, 'clean', 'd3_departments.csv'))
# df_sales_stores = pd.read_csv(os.path.join(resource_dir, 'clean', 'd3_stores.csv'))
# df_sales_states = pd.read_csv(os.path.join(resource_dir, 'clean', 'd3_states.csv'))
# df_sales_items = pd.read_csv(os.path.join(resource_dir, 'clean', 'd3_items.csv'))

# df_ecomm = pd.read_csv(os.path.join(resource_dir, 'clean', 'd7_ecomm.csv'))
# # df_ecomm_prod_cat = pd.read_csv(os.path.join(resource_dir, 'clean', 'd7_prod_cat.csv'))

# # df_ecomm_category_level1 = pd.read_csv(os.path.join(resource_dir, 'clean', 'd7_cat_lvl1.csv'))
# # df_ecomm_category_level2 = pd.read_csv(os.path.join(resource_dir, 'clean', 'd7_cat_lVl2.csv'))
# # df_ecomm_category_level3 = pd.read_csv(os.path.join(resource_dir, 'clean', 'd7_cat_lvl3.csv'))
# # df_ecomm_category_level4 = pd.read_csv(os.path.join(resource_dir, 'clean', 'd7_cat_lvl4.csv'))
# # df_ecomm_category_level5 = pd.read_csv(os.path.join(resource_dir, 'clean', 'd7_cat_lvl5.csv'))
# # df_ecomm_category_level6 = pd.read_csv(os.path.join(resource_dir, 'clean', 'd7_cat_lvl6.csv'))

## Step 4. Load data into PostgreSQL

In [76]:
# this code loops through the folder of cleaned .csv files and loads them to PostgreSQL
# this is over 20 faster than using sqlalchemy and df.to_sql for long tables

# files are read to memory using StringIO in the io package
# 'copy [table] from stdin' in PostgreSQL, which directly from memory on the local computer

folder_name = os.path.join(resource_dir, 'clean')

conn_host = 'otto.db.elephantsql.com'
conn_dbname = 'ofiglsqd'
conn_user = 'ofiglsqd'

# Ask the user for which database they want to use and which credentials to access it
# conn_host = input('host: ')
# conn_dbname = input('database: ')
# conn_user = input('username: ')
conn_pass = getpass.getpass(prompt='password: ')

# loop through .csv files in the output folder
for file in os.listdir(folder_name):

    print('\n\n' + str(datetime.utcnow()) + ' ' + str(file) + ' to be loaded')

    print(str(datetime.utcnow()) + ' reading file to dataframe...')
    
    # read .csv file into dataframe
    df = pd.read_csv(os.path.join(folder_name, file), na_values=['nan','NA','NaN'])
    
    print(str(datetime.utcnow()) + ' completed')
    
    print(df.info(memory_usage='deep'))
    
    # 
    with psycopg2.connect(host=conn_host, dbname=conn_dbname, user=conn_user, password=conn_pass) as conn:
        conn.autocommit = True

        table_name = file.split('.csv')[0].lower().replace('-','_')

        output = io.StringIO()

        print(str(datetime.utcnow()) + ' reading file to memory using StringIO...')

        df.to_csv(output, sep='|', header=False, index=False)
        output.seek(0)

        print(str(datetime.utcnow()) + ' completed')

        print(str(datetime.utcnow()) + ' generating the create table statement...')
        
        qry = pd.io.sql.get_schema(df, table_name, con=conn)

        qry = qry.replace('CREATE TABLE', 'CREATE TABLE IF NOT EXISTS')

        for key in df.columns:
            if pd.api.types.infer_dtype(df[key], skipna=True) == 'boolean':
                start = qry.find(key)
                end = start + qry[start:].find(',')
                print(start, end)
                qry = qry[:start] + key + '" BOOLEAN' + qry[end:]
        try:
            with conn.cursor() as cur:
                print(str(datetime.utcnow()) + ' completed')
                print(qry)
                
                print(str(datetime.utcnow()) + ' executing the create table statement...')
                cur.execute(qry)
                print(str(datetime.utcnow()) + ' completed')
                
                print(str(datetime.utcnow()) + ' loading table to database...')
                cur.copy_expert("""COPY %s FROM STDIN WITH (FORMAT csv, DELIMITER '|', QUOTE '"')""" % table_name, output)
                print(str(datetime.utcnow()) + ' completed')

        except Exception as e:
            print('Error:\n' + str(e))




2020-12-18 23:26:37.824655 d3_categories.csv to be loaded
2020-12-18 23:26:37.824655 reading file to dataframe...
2020-12-18 23:26:37.830652 completed
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cat_desc  3 non-null      object
 1   cat_id    3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 344.0 bytes
None
2020-12-18 23:26:38.632144 reading file to memory using StringIO...
2020-12-18 23:26:38.633144 completed
2020-12-18 23:26:38.633144 generating the create table statement...
2020-12-18 23:26:39.038207 completed
CREATE TABLE IF NOT EXISTS "d3_categories" (
"cat_desc" TEXT,
  "cat_id" INTEGER
)
2020-12-18 23:26:39.038207 executing the create table statement...
2020-12-18 23:26:39.128597 completed
2020-12-18 23:26:39.128597 loading table to database...
2020-12-18 23:26:39.285345 completed


2020-12-18 23:26:39.285345 d3_departments.cs

<ipython-input-76-da67d0f2315e>:51: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  qry = pd.io.sql.get_schema(df, table_name, con=conn)


2020-12-18 23:40:34.388265 completed
